<img src="Bilder/ost_logo.png" width="240"  align="right"/>
<div style="text-align: left"> <b> Applied Neural Networks | FS 2025 </b><br>
<a href="mailto:christoph.wuersch@ost.ch"> © Christoph Würsch, François Chollet </a> </div>
<a href="https://www.ost.ch/de/forschung-und-dienstleistungen/technik/systemtechnik/ice-institut-fuer-computational-engineering/"> Eastern Switzerland University of Applied Sciences OST | ICE </a>

[![Run in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ChristophWuersch/AppliedNeuralNetworks/blob/main/ANN06/6.1-CNN_auf_kleinen_Datensätzen_pytorch.ipynb)

# 5.2 Ein CNN von Grund auf mit einer kleinen Datenmenge trainieren

This notebook contains the code sample found in Chapter 5, Section 2 of [Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python?a_aid=keras&a_bid=76564dff).

Wenn Sie in einem beruflichen Umfeld maschinelles Sehen betreiben, werden Sie
in der Praxis häufig mit der Situation konfrontiert, dass für das Training eines
Bildklassifizierungsmodells nur wenige Daten zur Verfügung stehen. »Wenige«
kann hier einige Hundert, aber auch mehreren Zehntausend Bilder bedeuten. 

Als praktisches Beispiel betrachten wir die Klassifizierung von Bildern als Hunde oder
Katzen. Die Datenmenge besteht aus 4.000 Bildern (2.000 Hunde und 2.000 Katzen).
Wir verwenden 2.000 Bilder für das Training und jeweils 1.000 für die Validierung
bzw. das Testen.


In diesem Abschnitt werden wir eine grundlegende Strategie dazu erörtern, wie
wir die Aufgabe in Angriff nehmen können, ein neues Modell von Grund auf mit
den wenigen vorhandenen Daten zu trainieren. 

- Zunächst einmal werden wir ein kleines naives CNN **ohne Regularisierung** mit den 2.000 Trainingssamples trainieren, um einschätzen zu können, was sich erreichen lässt. Die Korrektklassifizierungsrate dieses Modells beträgt 71%. Hier ist die **Überanpassung** das entscheidende Problem. 


- Anschliessend wird die **Datenaugmentation** vorgestellt, ein leistungsfähiges Verfahren zur Abschwächung der Überanpassung beim maschinellen Sehen. Mit diesem Verfahren werden wir die Korrektklassifizierungsrate des NNs auf 82% verbessern können.


- Im darauffolgenden Abschnitt werden wir uns mit zwei weiteren elementaren Verfahren zur Anwendung des Deep Learnings auf kleine Datenmengen befassen: **Merkmalsextraktion** mit einem vortrainierten NN (damit werden wir eine Korrektklassifizierungsrate von 90 bis 96% erzielen) und **Feinabstimmung eines vortrainierten NNs** (hiermit werden wir schließlich eine Korrektklassifizierungsrate von 97% erreichen). 


Zusammengenommen bilden diese **drei Strategien**
- (a) **Training** eines kleinen Modells von Grund auf
- (b) **Merkmalsextraktion** mit einem vortrainierten Modell und 
- (c) **Feinabstimmung** eines vortrainierten Modells
die Grundlage für Ihren zukünftigen Werkzeugkasten für die Bildklassifizierung, wenn nur wenige Daten verfügbar sind.

## Die Bedeutung des Deep Learnings für Aufgaben mit kleinen Datenmengen

Mitunter heisst es, dass Deep Learning nur funktioniert, wenn sehr viele Daten
verfügbar sind. 
- Das stimmt zum Teil: Zu den grundlegenden Eigenschaften des *Deep Learnings* gehört die Fähigkeit, eigenständig interessante Merkmale in den Trainingsdaten aufzuspüren, ohne dass eine manuelle Merkmalserstellung erforderlich wäre.
- Das lässt sich jedoch nur erreichen, wenn viele Trainingssamples verfügbar sind. 
- Das trifft insbesondere auf Aufgaben zu, bei denen die Eingabedaten  sehr viele Dimensionen besitzen, wie es bei Bildern der Fall ist.

Was aber »viele Samples« konkret bedeutet, ist relativ und hängt von der Grösse und Tiefe des zu trainierenden NNs ab. So ist es beispielsweise nicht möglich, ein CNN mit nur einigen Dutzend Samples darauf zu trainieren, komplexe Aufgaben
zu lösen, allerdings können ein paar Hundert schon durchaus ausreichen, wenn
das Modell klein, gut regularisiert und die Aufgabe einfach ist. 



Da CNNs **lokale translationsinvariante Merkmale** erlernen, sind sie insbesondere für Aufgaben der
Sinneswahrnehmung sehr effizient. Ein CNN von Grund auf mit einer sehr kleinen
Datenmenge zu trainieren, wird trotz des relativen Mangels an Daten zu halbwegs
vernünftigen Ergebnissen führen, ohne dass eine Merkmalserstellung von
Hand erforderlich wäre.

Darüber hinaus sind Deep-Learning-Modelle naturgemäss gut wiederverwendbar:
- Sie können beispielsweise ein mit einer grossen Datenmenge trainiertes Bildklassifizierungs- oder Spracherkennungsmodell zur Lösung deutlich anderer Aufgaben verwenden, ohne grössere Änderungen vornehmen zu müssen. 
- Für das maschinelle Sehen stehen inzwischen viele vortrainierte Modelle (die für gewöhnlich mit der ImageNet-Datensammlung trainiert wurden) öffentlich zum Herunterladen bereit und können genutzt werden, um mit sehr wenigen Daten leistungsfähige Modelle für das maschinelle Sehen zu entwickeln. 

Genau das werden wir im nächsten Abschnitt tun. Aber zunächst einmal werden die Daten benötigt.

## Daten herunterladen

Der Datensatz, den wir hier verwenden werden, ist kein Bestandteil von Keras.
Er wurde im Rahmen eines Wettbewerbs für maschinelles Sehen Ende 2013 von
Kaggle zur Verfügung gestellt. Damals waren CCNs noch nicht in aller Munde.
Sie können die Datenmenge unter http://www.kaggle.com/c/dogs-vs-cats/
data herunterladen. (Dazu müssen Sie ein Konto einrichten, falls Sie noch keins besitzen – aber keine Sorge, der Vorgang ist ganz einfach.)

Bei den Bildern handelt es sich um farbige JPEG-Dateien mittlerer Auflösung.
Die folgende Abbildung zeigt einige Beispiele.

<img src="Bilder/cats_vs_dogs_samples.jpg" width="640"  align="center"/>

*Abbbildung 5.9 Beispiele der Bilder von Hunden und Katzen. Die Grösse der Bilder wurde nicht
geändert: Die Samples sind von unterschiedlicher Grösse, liegen in verschiedenen
Formaten vor usw.*

## Kaggle

Dass der Kaggle-Wettbewerb zur Unterscheidung von Hunden und Katzen 2013
von Teilnehmern gewonnen wurde, die CNNs nutzten, war keine Überraschung.
Die besten Teilnehmer erreichten eine Korrektklassifizierungsrate von bis zu
95%. Bei diesem Beispiel werden wir (im nächsten Abschnitt) eine ähnlich gute
Korrektklassifizierungsrate erzielen, obwohl wir zum Trainieren der Modelle nur
10% der Daten verwenden werden, die den Wettbewerbsteilnehmern zur Verfügung
standen. Diese Datenmenge enthält 25.000 Bilder von Hunden und Katzen
(jeweils 12.500 von jeder Klasse) und ist 543 MB gross (komprimiert). Nach dem
Herunterladen und Entpacken können Sie eine neue aus drei Teilmengen bestehende
Datenmenge erzeugen: eine Trainingsdatenmenge mit jeweils 1.000
Samples beider Klassen sowie eine Validierungs- und eine Testdatenmenge mit
jeweils 500 Samples beider Klassen.

Nachfolgend der dazu erforderliche Code.

In [ ]:
import os, shutil


In [ ]:
def createDir(train_dir):
    if not os.path.exists(train_dir):
        os.mkdir(train_dir)
    else:
        print('already existing: %s' % train_dir)


In [ ]:
# The path to the directory where the original
# dataset was uncompressed
original_dataset_dir = 'E:/teaching/ANN/datasets/kaggle_original_data/train'


# The directory where we will
# store our smaller dataset
base_dir = 'E:/teaching/ANN/datasets/cats_and_dogs_small'

createDir(base_dir)


#### Trainings Datensatz

In [ ]:
# Directories for our training,
# validation and test splits
train_dir = os.path.join(base_dir, 'train')
createDir(train_dir)

validation_dir = os.path.join(base_dir, 'validation')
createDir(validation_dir)

test_dir = os.path.join(base_dir, 'test')
createDir(test_dir)

# Directory with our training cat pictures
train_cats_dir = os.path.join(train_dir, 'cats')
createDir(train_cats_dir)
    
# Directory with our training dog pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')
createDir(train_dogs_dir)


#### Validation Datensatz:

In [ ]:
# Directory with our validation cat pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')
createDir(validation_cats_dir)

# Directory with our validation dog pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
createDir(validation_dogs_dir)


#### Test Datensatz

In [ ]:

# Directory with our test cat pictures
test_cats_dir = os.path.join(test_dir, 'cats')
createDir(test_cats_dir)

# Directory with our test dog pictures
test_dogs_dir = os.path.join(test_dir, 'dogs')
createDir(test_dogs_dir)


In [ ]:
print(original_dataset_dir)
print(train_cats_dir)


In [ ]:

# Copy first 1000 cat images to train_cats_dir
fnames = ['cat.{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_cats_dir, fname)

    shutil.copyfile(src, dst)

# Copy next 500 cat images to validation_cats_dir
fnames = ['cat.{}.jpg'.format(i) for i in range(1000, 1500)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_cats_dir, fname)
    shutil.copyfile(src, dst)
    
# Copy next 500 cat images to test_cats_dir
fnames = ['cat.{}.jpg'.format(i) for i in range(1500, 2000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_cats_dir, fname)

    shutil.copyfile(src, dst)
    


In [ ]:
# Copy first 1000 dog images to train_dogs_dir
fnames = ['dog.{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_dogs_dir, fname)
    shutil.copyfile(src, dst)
    
# Copy next 500 dog images to validation_dogs_dir
fnames = ['dog.{}.jpg'.format(i) for i in range(1000, 1500)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_dogs_dir, fname)
    shutil.copyfile(src, dst)
    
# Copy next 500 dog images to test_dogs_dir
fnames = ['dog.{}.jpg'.format(i) for i in range(1500, 2000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_dogs_dir, fname)
    shutil.copyfile(src, dst)


Zur Überprüfung der Vollständigkeit der Daten ermitteln wir die Anzahl der Bilder
in den Teilmengen (Trainings-, Validierungs- und Testdatenmenge):

In [ ]:
print('total training cat images:', len(os.listdir(train_cats_dir)))


In [ ]:
print('total training dog images:', len(os.listdir(train_dogs_dir)))


In [ ]:
print('total validation cat images:', len(os.listdir(validation_cats_dir)))


In [ ]:
print('total validation dog images:', len(os.listdir(validation_dogs_dir)))


In [ ]:
print('total test cat images:', len(os.listdir(test_cats_dir)))


In [ ]:
print('total test dog images:', len(os.listdir(test_dogs_dir)))


Es sind also tatsächlich 2.000 Trainingsbilder, 1.000 Validierungsbilder und 1.000
Testbilder vorhanden. Jede Teilmenge enthält dieselbe Anzahl Samples beider Klassen:
Hierbei handelt es sich um eine *ausgewogene Binärklassifizierungsaufgabe* (engl. *balanced classes*).

## Architektur des NN erzeugen

Im letzten Beispiel haben wir für die Klassifizierung der MNIST-Ziffern ein kleines
CNN erstellt. NNs dieser Art sollten Ihnen also bereits vertraut sein. Wir verwenden
hier die gleiche allgemeine Struktur: 
- Das CNN besteht aus einem Stapel, in dem sich `Conv2D`- (mit `relu`-Aktivierung) und `MaxPooling2D`-Layer abwechseln.
- Da wir es hier jedoch mit grösseren Bildern und einer komplexeren Aufgabenstellung zu tun haben, verwenden wir ein dementsprechend grösseres NN: 
- Es besitzt eine zusätzliche aus einem `Conv2D`- und einem `MaxPooling2D`-Layer bestehende Stufe. Diese soll zum einen die Kapazität des CNNs erhöhen und zum anderen die Grösse der Feature-Map weiter reduzieren, damit sie nicht allzu gross ist, wenn der `Flatten`-Layer erreicht wird. 
- Da hier eingangs Eingaben der Grösse 150 × 150 verwendet werden (eine mehr oder weniger willkürliche Wahl), ergeben sich unmittelbar vor dem `Flatten`-Layer Feature-Maps der Grösse 7 × 7.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pytorch_lightning as pl
from torch.utils.data import DataLoader, random_split
from torchvision import transforms, datasets
from pytorch_lightning.loggers import CSVLogger
import matplotlib.pyplot as plt
import pandas as pd


In [ ]:
# Define CNN model
class CNNModel(pl.LightningModule):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        self.conv1 = torch.nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=0)
        self.pool = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=0)
        self.conv3 = torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=0)
        self.conv4 = torch.nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=0)
        
        self.flatten = torch.nn.Flatten()
        self.dropout = torch.nn.Dropout(0.5)
        self.fc1 = torch.nn.Linear(128 * 7 * 7, 512)  # Adjusted for input size
        self.fc2 = torch.nn.Linear(512, 1)
        self.sigmoid = torch.nn.Sigmoid()
        
        self.criterion = torch.nn.BCEWithLogitsLoss()
        
    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = self.pool(x)
        x = torch.relu(self.conv2(x))
        x = self.pool(x)
        x = torch.relu(self.conv3(x))
        x = self.pool(x)
        x = torch.relu(self.conv4(x))
        x = self.pool(x)
        x = self.flatten(x)
        x = self.dropout(x)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        y = y.unsqueeze(1).float()  # Ensure correct shape
        loss = self.criterion(y_hat, y)
        acc = ((y_hat > 0.5) == y).float().mean()
        self.log("train_loss", loss, prog_bar=True)
        self.log("train_acc", acc, prog_bar=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        y = y.unsqueeze(1).float()
        loss = self.criterion(y_hat, y)
        acc = ((y_hat > 0.5) == y).float().mean()
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", acc, prog_bar=True)
        return loss
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3)



In [ ]:
32*(3*3*3+1)


In [ ]:
32*64*(3*3)+64


In [ ]:
64*128*(3*3)+128


Die Anzahl der Kanäle der Feature-Maps nimmt beim Durchlaufen des NNs
nach und nach zu (sie wächst von 32 auf 128), während die Größe allmählich
sinkt (von 150 × 150 auf 7 × 7). Dieses Verhalten werden Sie bei fast allen CNNs
beobachten.

In [ ]:
from pytorch_lightning.utilities.model_summary import summarize

model = CNNModel()
print(summarize(model, max_depth=3))  # max_depth controls how much detail to show


Die gegebene Eingangsgröße für die Daten ist:

- **Bilder**: `[batch_size, channels, height, width]` → `[20, 3, 150, 150]`
- **Labels**: `[batch_size]` → `[20]`

Das aktuelle CNN-Architektur-Design berücksichtigt jedoch nicht die tatsächliche Größe der Feature Maps, die nach den Faltungs- und Pooling-Schichten entstehen. Dies führt zu einer falschen Anzahl von Eingängen in die Fully Connected (FC) Layer.

Die Architektur besteht aus mehreren **Faltungs- und Max-Pooling-Schichten**, die die räumlichen Dimensionen der Bilder verändern. Die Berechnung erfolgt wie folgt:

1. **Conv2D (3 → 32, Kernel: 3x3, kein Padding) → MaxPool(2x2)**

   - Eingangsgröße: `(150, 150, 3)`
   - Nach der Faltung: `(148, 148, 32)` (weil 3x3-Filter ohne Padding genutzt wird)
   - Nach MaxPool(2x2): `(74, 74, 32)`

2. **Conv2D (32 → 64, Kernel: 3x3) → MaxPool(2x2)**

   - Nach der Faltung: `(72, 72, 64)`
   - Nach MaxPool(2x2): `(36, 36, 64)`

3. **Conv2D (64 → 128, Kernel: 3x3) → MaxPool(2x2)**

   - Nach der Faltung: `(34, 34, 128)`
   - Nach MaxPool(2x2): `(17, 17, 128)`

4. **Conv2D (128 → 128, Kernel: 3x3) → MaxPool(2x2)**

   - Nach der Faltung: `(15, 15, 128)`
   - Nach MaxPool(2x2): `(7, 7, 128)`

## Notwendige Anpassung der Fully Connected Layer

Die  Berechnung ergibt eine **Feature-Map-Größe von 7 × 7 × 128 = 6272**. Daher muss die Fully Connected Layer angepasst werden:


In [ ]:
from torchinfo import summary

summary(model, input_size=(20, 3, 150, 150))  # Batch size of 1, 3 color channels, 150x150 image


Da es sich hier um eine **Binärklassifizierungsaufgabe** handelt, endet das NN mit
einer einzelnen Einheit (einem Dense-Layer der Grösse 1) und einer *sigmoid-Aktivierung*.
Diese Einheit gibt die Wahrscheinlichkeit dafür an, dass das NN die eine
oder die andere der beiden Klasse erkannt hat.

## Optimierer und Kostenfunktion (loss)

Bei der Kompilierung kommt wie üblich der `Adam`-Optimierer zum Einsatz. Da
das NN mit einer sigmoiden Einheit endet, wird als Kosten- oder Verlustfunktion die **binäre
Kreuzentropie** verwendet.


## Datenvorverarbeitung (data preprocessing)

Wie Sie inzwischen wissen, sollten die Daten bei der Vorverarbeitung in Fliesskommazahltensoren
umgewandelt werden, bevor sie in das NN eingespeist werden.
Die Daten sind in Form von JPEG-Dateien auf der Festplatte gespeichert,
daher sind für die Verarbeitung durch das NN die folgenden Schritte erforderlich:
1. Lesen Sie die Bilddateien ein.
2. **Wandeln** Sie die JPEG-Dateien in **RGB**-Pixelwerte um.
3. **Konvertieren** Sie die RGB-Pixelwerte in **Fliesskommazahl**-Tensoren (`float`).
4. **Skalieren** Sie die Pixelwerte aus dem Bereich von 0 bis 255 neu, sodass sie im Intervall `[0, 1]` liegen. (Wie Sie wissen, sind standardisierte Eingabewerte für NNs besser geeignet.)


Das mag mühsam erscheinen, aber glücklicherweise bietet Keras verschiedene
Hilfsprogramme, die diese Aufgaben automatisch erledigen können. Zu diesem
Zweck gibt es ein Modul mit Hilfsprogrammen zur Bildverarbeitung, das Sie
unter keras.preprocessing.image finden. Es enthält insbesondere die Klasse
ImageDataGenerator, die es ermöglicht, auf der Festplatte gespeicherte Bilddateien
automatisch in einen Stapel von Tensoren der vorverarbeiteten Bilder umzuwandeln.
Diese Klasse wird hier verwendet.

In [ ]:
print(validation_dir)
print(train_dir)


In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define transformations similar to ImageDataGenerator
transform = transforms.Compose([
    transforms.Resize((150, 150)),  # Resize images to 150x150
    transforms.ToTensor(),          # Convert images to PyTorch tensors
    transforms.Normalize((0.5,), (0.5,))  # Normalize (similar to rescaling 1./255)
])

# Load datasets
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
validation_dataset = datasets.ImageFolder(root=validation_dir, transform=transform)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=20, shuffle=False)


## Python-Generatoren 

- Ein Python-Generator ist ein Objekt, das als **Iterator** fungiert. 
- Sie können ein solches Objekt zusammen mit dem Operator `for ... in` verwenden. 
- Zum Erzeugen eines Generators dient der `yield`-Operator.

Hier ist ein Beispiel für einen Generator, der Integer liefert:

In [ ]:
def generator():
    i = 0
    while True:
        i += 1
        yield i

for item in generator():
    print(item)
    if item > 4:
        break


Betrachten Sie die Ausgabe eines dieser Generatoren: 
- Er liefert Stapel von 150 × 150 Pixel grossen RGB-Bildern (Shape (20, 150, 150, 3)) und binären Klassenbezeichnungen (Shape (20,)). 
- Jeder dieser Stapel enthält 20 Samples.
- Beachten Sie hier, dass der Generator die Bilder im Zielverzeichnis in einer Endlosschleife durchläuft. 
- Deshalb müssen Sie die Schleife irgendwann mit der break-Anweisung anhalten:

In [ ]:
# Iterate through the DataLoader for one batch
for data_batch, labels_batch in train_loader:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break


Nun passen wir das Modell mithilfe des **Generators** an die Daten an. 

- Dazu verwenden wir die Methode`fit()`, welche auch für Datengeneratoren wie diesen unterstützt.
- Die Methode erwartet als erstes Argument einen Python-Generator, der auf unbestimmte Zeit Stapel von Eingaben und Zielwerten liefert, wie es hier der Fall ist. 
- Da die Daten auf unbestimmte Zeit erzeugt werden, muss das Keras-Modell wissen, wie viele Samples durch den Generator abgerufen werden sollen, um eine Epoche abzuschliessen. 
- Dazu dient das Argument `steps_per_epoch`: Nach dem Abruf der `steps_per_epoch` Stapel durch den Generator – also nachdem die `steps_per_epoch` Gradientenabstiegsschritte erledigt wurden –, fährt die `fit()`-Methode mit der nächsten Epoche fort. 


Beim Aufruf von `fit()` können Sie das Argument `validation_data` übergeben. 
- An dieser Stelle ist es wichtig, darauf zu achten, dass dieses Argument zwar ein Datengenerator sein darf, es könnte jedoch auch ein Tupel von `Numpy`-Arrays sein. 
- Wenn Sie als `validation_data` einen Generator übergeben, sollte er auf unbestimmte Zeit Stapel von Validierungsdaten liefern. 
- In diesem Fall sollten Sie zudem dem Argument `validation_steps` einen Wert zuweisen, der festlegt, wie viele Stapel der Generator für die Bewertung abrufen soll.

Im vorliegenden Fall bestehen die Stapel aus 20 Samples, daher sind 100 Stapel nötig, um die 2.000 Samples zu verarbeiten.
Wir trainieren für 30 Epochen.

In [ ]:
# Train model
logger = CSVLogger("logs", name="cnn_model")
model = CNNModel()
trainer = pl.Trainer(max_epochs=10, accelerator="auto", logger=logger)
trainer.fit(model, train_loader, validation_loader)


Es empfiehlt sich, das Modell nach dem Training stets abzuspeichern:

In [ ]:
torch.save(model.state_dict(), "cats_and_dogs_small_1.pth")


`model.state_dict()` speichert nur die Gewichte des Modells (empfohlen für PyTorch-Modelle).
Sie können das Modell später laden mit:

In [ ]:
model = CNNModel()  # Initialize the model
model.load_state_dict(torch.load("cats_and_dogs_small_1.pth"))
model.eval()  # Set to evaluation mode


Wenn Sie die gesamte Modellarchitektur und die Gewichte zusammen speichern möchten, verwenden Sie:

Nun geben wir die Diagramme der Korrektklassifizierungsrate und der Verlustfunktion
beim Training aus.

In [ ]:
# Load training logs and plot
log_path = "logs/cnn_model/version_3/metrics.csv"
df = pd.read_csv(log_path)
df.head()
df=df.groupby('epoch').mean()


In [ ]:
def plot_history(df):   
    plt.figure(figsize=(10,5))
    plt.plot(df["step"], df["train_loss"],'.-', label="Train Loss")
    plt.plot(df["step"], df["val_loss"],'.-.', label="Validation Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.title("Training and Validation Loss")
    plt.grid()
    plt.show()

    plt.figure(figsize=(10,5))
    plt.plot(df["step"], df["train_acc"], '.-',label="Train Accuracy")
    plt.plot(df["step"], df["val_acc"], '.-.',label="Validation Accuracy")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.title("Training and Validation Accuracy")
    plt.grid()
    plt.show()


In [ ]:
plot_history(df)


Die beiden Diagramme sind für eine **Überanpassung (overfitting)** charakteristisch. 
- Die Korrektklassifizierungsrate nimmt beim Training im Laufe der Zeit annähernd linear zu, bis sie schließlich fast 100% erreicht, während sie bei der Validierung nicht über 70% bis 72% hinauskommt. 
- Die Verlustfunktion erreicht bei der Validierung schon nach der fünften Epoche ihr Minimum und ändert sich während der folgenden Epochen kaum. 
- Die Verlustfunktion beim Training hingegen nimmt annähernd linear ab, bis sie fast null erreicht.

Da hier nur relativ wenige Trainingssamples vorliegen (2.000), werden Sie der Überanpassung die größte Beachtung schenken müssen. Sie kennen inzwischen ja schon einige der Methoden zum Abschwächen der Überanpassung, wie das
**Dropout-Verfahren** oder die **L2-Regularisierung**. 

Nun werden wir ein weiteres für das maschinelle Sehen geeignetes Verfahren einsetzen, das fast überall bei der
Bildverarbeitung mit Deep-Learning-Modellen verwendet wird: **die Datenaugmentation (data augmentation)**.

## Data Augmentation

- Die Überanpassung wird dadurch verursacht, dass zu wenige Samples verfügbar sind, sodass es nicht möglich ist, ein Modell zu trainieren, das sich gut für neue Daten verallgemeinern lässt. 
- Wären unbegrenzt Daten verfügbar, würde Ihr Modell sämtliche Aspekte der vorliegenden Datenverteilung kennen: Es würde nie zu einer Überanpassung kommen. 
- Die **Datenaugmentation** verfolgt den Ansatz, anhand der vorhandenen Trainingssamples zusätzliche Daten zu erstellen, indem die Datensammlung durch neue Samples erweitert wird, die durch eine Reihe **zufälliger Transformationen** erzeugt werden, die glaubwürdig aussehende Bilder liefern. 
- Das Ziel ist hier, dass Ihr Modell beim Training niemals zwei völlig identische Bilder zu sehen bekommt. Das erleichtert es dem Modell, weitere Aspekte der Daten zu erkennen, und verbessert die Verallgemeinerungsfähigkeit.

In Keras kann dieses Ziel durch die Konfiguration verschiedener zufälliger Transformationen erreicht werden, die auf die von der `ImageDataGenerator`-Instanz eingelesenen Bilder angewendet werden. Betrachten wir zunächst einmal ein Beispiel:

In [ ]:

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define transformations with data augmentation for training set
train_transform = transforms.Compose([
    transforms.Resize((150, 150)),  # Resize images to 150x150
    transforms.RandomRotation(40),  # Random rotation within 40 degrees
    transforms.RandomAffine(degrees=0, shear=0.2),  # Shear transformation
    transforms.RandomResizedCrop(150, scale=(0.8, 1.2)),  # Zoom effect
    transforms.RandomHorizontalFlip(),  # Horizontal flipping
    transforms.RandomAffine(degrees=0, translate=(0.2, 0.2)),  # Width & height shift
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize((0.5,), (0.5,))  # Normalize
])

# Transformations for validation set (no augmentation)
validation_transform = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load datasets
train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transform)
validation_dataset = datasets.ImageFolder(root=validation_dir, transform=validation_transform)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=32, shuffle=False)




### **Erklärung der Augmentationen**

| Augmentation | Beschreibung |
|-------------|-------------|
| `transforms.RandomRotation(40)` | Dreht das Bild zufällig um bis zu ±40 Grad. |
| `transforms.RandomAffine(degrees=0, shear=0.2)` | Wendet eine Schertransformation an. |
| `transforms.RandomResizedCrop(150, scale=(0.8, 1.2))` | Wendet einen Zoom-Effekt an. |
| `transforms.RandomHorizontalFlip()` | Spiegelt das Bild zufällig horizontal. |
| `transforms.RandomAffine(degrees=0, translate=(0.2, 0.2))` | Verschiebt das Bild zufällig um 20 % der Bildgröße. |
| `transforms.ToTensor()` | Konvertiert Bilder in Tensoren. |
| `transforms.Normalize((0.5,), (0.5,))` | Normalisiert die Pixelwerte. |

Diese Konfiguration ahmt `ImageDataGenerator` in PyTorch nach und stellt eine robuste Datenaugmentation für das Training sicher. 🚀

In [ ]:
import matplotlib.pyplot as plt
from torchvision import transforms
from PIL import Image

# Define the same data augmentation transformations
transform = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.RandomRotation(40),
    transforms.RandomAffine(degrees=0, shear=0.2),
    transforms.RandomResizedCrop(150, scale=(0.8, 1.2)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomAffine(degrees=0, translate=(0.2, 0.2)),
    transforms.ToTensor()
])

# Get a sample image path
fnames = [os.path.join(train_cats_dir, fname) for fname in os.listdir(train_cats_dir)]
img_path = fnames[1]  # Select one image

# Load and preprocess the image
img = Image.open(img_path).convert("RGB")
img = img.resize((150, 150))

# Convert image to tensor
img_tensor = transforms.ToTensor()(img)
img_tensor = img_tensor.unsqueeze(0)  # Reshape to (1, C, H, W)

# Generate augmented images
fig, axes = plt.subplots(1, 4, figsize=(12, 6))
i = 0
for _ in range(4):
    augmented_img = transform(img)
    axes[i].imshow(transforms.ToPILImage()(augmented_img))
    axes[i].axis("off")
    i += 1

plt.show()


Wenn Sie ein neues NN mit dieser Konfiguration der Datenaugmentation trainieren,
wird es niemals zwei identische Bilder zu sehen bekommen. 
- Die Bilder, die es sieht, sind dessen ungeachtet eng miteinander verwoben, denn sie wurden ja anhand nur weniger ursprünglicher Bilder erstellt. Sie können auf diese Weise keine neuen Informationen erzeugen, sondern lediglich die vorhandenen Informationen anders miteinander verknüpfen. 
- Daher reicht dieses Verfahren womöglich nicht aus, um die Überanpassung komplett loszuwerden. 
- Um die Überanpassung weiter abzuschwächen, fügen wir dem Modell unmittelbar vor dem vollständig verbundenen Klassifizierer einen **`Dropout`-Layer** hinzu.

**NB: Die Validierungsdaten sollen nicht transformiert werden!**


In [ ]:
# Train model
logger = CSVLogger("logs", name="cnn_model2")
model2 = CNNModel()
trainer = pl.Trainer(max_epochs=30, accelerator="auto", logger=logger)
trainer.fit(model2, train_loader, validation_loader)


Und nun trainieren wird das CNN mit Datenaugmentation und Dropout-Layer.

Speichern Sie das Modell – Sie werden es in Abschnitt 5.4 erneut verwenden.

In [ ]:
torch.save(model.state_dict(), "cats_and_dogs_small_2.pth")
#model2 = CNNModel()  # Initialize the model
#model2.load_state_dict(torch.load("cats_and_dogs_small_1.pth"))
#model2.eval()  # Set to evaluation mode


In [ ]:
# Load training logs and plot
log_path = "logs/cnn_model2/version_1/metrics.csv"
dg = pd.read_csv(log_path)
dg.head()
dg=dg.groupby('epoch').mean()
plot_history(dg)


Wir geben wieder die Diagramme aus (siehe Abbildung 5.12 und Abbildung 5.13).


Dank *Datenaugmentation und Dropout-Verfahren* kommt es nicht mehr zu einer *Überanpassung*: Die Kurven beim Training und bei der Validierung verlaufen sehr ähnlich. Sie erzielen jetzt eine Korrektklassifizierungsrate von 82%, eine (relative)
Verbesserung um 15% im Vergleich zum Modell ohne Regularisierung.

- Durch den Einsatz weiterer *Regularisierungsverfahren* und die Abstimmung der Parameter des NNs (etwa die Anzahl der Filter pro Layer oder die Anzahl der Layer im NN) kann sogar eine noch höhere Korrektklassifizierungsrate von bis zu 87% erreicht werden. 
- Es dürfte sich jedoch als schwierig erweisen, allein durch das Training des CNNs von Grund auf eine höhere Korrektklassifizierungsrate zu erzielen, weil nur so wenige Daten verfügbar sind. 
- Der nächste Schritt zur Verbesserung der Korrektklassifizierungsrate bei dieser Aufgabe ist der Einsatz eines vortrainierten Modells, auf das sich die beiden folgenden Abschnitte konzentrieren.


## Where's the intelligence?

- Was emuliert die Data Augmentation?
- Diskutieren Sie mit Ihren Peers, welche Regularisierungsverfahren für CNN angewendet werden können?
